In [1]:
import torchvision
import cv2
import torch
import matplotlib.pyplot as plt
from PIL import Image
import warnings
warnings.filterwarnings('ignore')
from torchvision import transforms as T

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Load the pre-trained Faster R-CNN model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True).to(device)

# Set the model to evaluation mode
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [4]:
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

In [5]:
def get_prediction(img, threshold):
  # img = Image.open(img_path) # Load the image
  transform = T.Compose([T.ToTensor()]) # Defing PyTorch Transform
  img = transform(img).to(device) # Apply the transform to the image
  pred = model([img]) # Pass the image to the model
  pred_class = [COCO_INSTANCE_CATEGORY_NAMES[i] for i in list(pred[0]['labels'].cpu().numpy())] # Get the Prediction Score
  pred_boxes = [[(i[0], i[1]), (i[2], i[3])] for i in list(pred[0]['boxes'].detach().cpu().numpy())] # Bounding boxes
  pred_score = list(pred[0]['scores'].detach().cpu().numpy())
  pred_t = [pred_score.index(x) for x in pred_score if x > threshold][-1] # Get list of index with score greater than threshold.
  pred_boxes = pred_boxes[:pred_t+1]
  pred_class = pred_class[:pred_t+1]
  return pred_boxes, pred_class

In [6]:
def object_detection_api(img, threshold=0.5, rect_th=3, text_size=1, text_th=1):
  boxes, pred_cls = get_prediction(img, threshold) # Get predictions
  # img = cv2.imread(img_path) # Read image with cv2
  # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Convert to RGB
  for i in range(len(boxes)):
    if(pred_cls[i]=="car"):
      # print(pred_cls[i])
      # print(boxes[i][0])
      # print(boxes[i][1])
      cv2.rectangle(img, (int(boxes[i][0][0]), int(boxes[i][0][1])), (int(boxes[i][1][0]), int(boxes[i][1][1])),color=(0, 255, 0), thickness=rect_th) # Draw Rectangle with the coordinate
      cv2.putText(img,pred_cls[i], (int(boxes[i][0][0]), int(boxes[i][0][1])),  cv2.FONT_HERSHEY_SIMPLEX, text_size, (0,255,0),thickness=text_th) # Write the prediction class

  return img

#   plt.figure(figsize=(20,30)) # display the output image
#   plt.imshow(img)
#   plt.xticks([])
#   plt.yticks([])
#   plt.show()

In [7]:
# object_detection_api('cars.jpg', threshold=0.8)
# plt.savefig('result.png')

In [8]:
cap = cv2.VideoCapture('../mp4/Test 1 .mp4')

In [9]:
text_size=1
text_th=1

In [10]:
# while cap.isOpened():
#     # Read the next frame
#     ret, frame = cap.read()

#     # Stop if the video has ended
#     if not ret:
#         break

#     # Convert the frame to a tensor and move it to the GPU
#     # tensor = torch.from_numpy(frame).permute(2, 0, 1).float().to(device)
#     transform = torchvision.transforms.ToTensor()
#     tensor = transform(frame).to(device)
#     # Make a forward pass through the model
#     with torch.no_grad():
#         outputs = model([tensor])

#     # Extract the bounding boxes and labels for the detected cars
#     boxes = outputs[0]['boxes'].cpu()
#     labels = outputs[0]['labels'].cpu()
#     indices = torch.argwhere(outputs[0]['scores'].cpu() > 0.8)

#     # Draw the bounding boxes on the frame
#     if (len(indices) != 0):
#         for i in range(len(indices)):
#             if (labels[indices[i]] == 3):
#                 x1, y1, x2, y2 = boxes[indices[i]].numpy().astype("int")[0]
#                 class_name = "car"
#                 cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 1)
#                 cv2.putText(frame, class_name, (x1, y1),  cv2.FONT_HERSHEY_SIMPLEX, text_size, (0,255,0),thickness=text_th) # Write the prediction class

#     # Display the frame with the detected cars
#     cv2.imshow('frame', frame)

#     # Wait for a key press to move to the next frame
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Release the video capture and close the window
# cap.release()
# cv2.destroyAllWindows()

In [11]:
while cap.isOpened():
    # Read the next frame
    ret, frame = cap.read()

    if not ret:
        break
    
    frame = object_detection_api(frame, threshold=0.8)
    cv2.imshow('frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()